In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 5.5 MB/s eta 0:00:00


In [4]:
!pip install -U bert-score transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 64.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.3
    Uninstalling transformers-4.57.3:
      Successfully uninstalled transformers-4.57.3


In [5]:
# ==========================================
# 환경 설정 및 라이브러리 설치
# ==========================================
#!pip uninstall -y torch torchvision torchaudio #import torch안되서 추가한것
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install git+https://github.com/SKT-AI/KoBART#egg=kobart
!pip install transformers==4.37.2 accelerate pandas tqdm
!pip install transformers torch pandas tqdm
!pip install -q jamo

import os
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from kobart import get_kobart_tokenizer, get_pytorch_kobart_model
from transformers import BartForConditionalGeneration, AdamW
from tqdm import tqdm
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from google.colab import drive

Looking in indexes: https://download.pytorch.org/whl/cu121
  Cloning https://github.com/SKT-AI/KoBART to /tmp/pip-install-l6qw_nef/kobart_eb6e28af946b459a80b517523f6b11f5
  Running command git clone --filter=blob:none --quiet https://github.com/SKT-AI/KoBART /tmp/pip-install-l6qw_nef/kobart_eb6e28af946b459a80b517523f6b11f5
  Resolved https://github.com/SKT-AI/KoBART to commit 79ff1d76ed47550695a05f4cd923642d754b0929
  Preparing metadata (setup.py) ... done
  Created wheel for kobart: filename=kobart-0.5.2-py3-none-any.whl size=8635 sha256=581dd5e527efc7ba1668198e81aa03259f649cf6e04eecdc56c6d4720fd634f0
  Stored in directory: /tmp/pip-ephem-wheel-cache-8wvbor8l/wheels/f5/70/d7/ae83146ce76b253c2ade2970070d4ae8029b4a2b241f13c3f5
Successfully built kobart
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 130.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 96.1 MB/s eta 0:00:00
  

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ 사용 중인 디바이스: {device}")

✅ 사용 중인 디바이스: cuda


In [7]:
import re
from jamo import h2j, j2hcj

def to_jamo_string(s: str) -> str:
    s = "" if s is None else str(s)
    out = []
    for ch in s:
        # 한글 음절(가-힣)만 자모로 분해
        if "가" <= ch <= "힣":
            out.append(j2hcj(h2j(ch)))
        else:
            out.append(ch)
    # 자모를 너무 붙여놓으면 토크나이저가 더 난리나는 경우가 있어서, 사이에 공백을 넣는 버전도 있음
    # 기본은 붙여서 유지. 필요하면 아래 한 줄로 바꿔서 실험해도 됨
    # return " ".join("".join(out))
    return "".join(out)

# 선택: 괄호/특수 토큰 등은 그대로 유지하고 싶으면 정규식으로 보호 처리 가능
# 지금은 단순하게 한글 음절만 분해

In [8]:
path1 = '/content/drive/MyDrive/멋사 NLP 자연어처리/팀프로젝트_1/데이터셋/noise_1(normal,10k).csv'
path2 = '/content/drive/MyDrive/멋사 NLP 자연어처리/팀프로젝트_1/데이터셋/new_train_normal_only.csv'

print("📂 데이터를 불러오는 중입니다...")

if all(os.path.exists(p) for p in [path1, path2]):
    df1 = pd.read_csv(path1, encoding='utf-8')[['input', 'output']]
    df2 = pd.read_csv(path2, encoding='utf-8')[['input', 'output']]

    df = pd.concat([df1, df2], axis=0).reset_index(drop=True)

    print("✅ 데이터 병합 완료")
    print("총 데이터 수:", len(df))
    print("구성 비율:")
    print(" - df1:", len(df1))
    print(" - df2:", len(df2))

else:
    missing = [p for p in [path1, path2] if not os.path.exists(p)]
    raise FileNotFoundError(f"❌ 파일이 없습니다: {missing}")

📂 데이터를 불러오는 중입니다...
✅ 데이터 병합 완료
총 데이터 수: 16169
구성 비율:
 - df1: 11263
 - df2: 4906


In [9]:
df["input"] = df["input"].apply(to_jamo_string)
# df["output"]은 그대로 둠

In [10]:
print("원문:", df.loc[0, "input"])
print("자모:", to_jamo_string(df.loc[0, "input"]))
print("정답:", df.loc[0, "output"])

원문: ㅃㅕㄹ ㅎㅑㄴ ㅋㅐㅋㄸㅛㄶ ㅇㅑㄲㅏㅂㄸㅑㄵ. ㅇㅙ ㅆㅏㅎㄹㅏㅁㄸㅡㅄㄹㅓ ㅃㅕㄹ 1ㄲㅐㄹㅡㄹ ㅉㅠㅆㄴㅡㄴㅉㅣ ㄲㅕㄲㅕㅃㅛㄴ ㅆㅑㄹㅏㅁㅡㄹㅛㅆㅅㅕㄳ ㅁㅑㄹㄹㅗ ㅆㅕㄹㅁㅕㅇㅎㅑㅉㅑㄴㅓ ㄸㅐㅅㄱㅡㄹㄹㅗㄴㅡㄴ ㄴㅓㅁㅠ ㄲㅓㄹㄲㅛㄵ... ㅇㅏㅁㅠㄷㅡㄴ ㄸㅜ ㅃㅕㄴ ㄸㅏㅆㅏㅆ ㄲㅑㄲㅓㄹ ㅆㅓㅀㅇㅡㄴ ㄲㅛㅅ. ㅋㅐㅁㅂㅣㅇㅡㄹ 20ㅇㅕ ㄴㅕㄴ ㄸㅏㄴㅕㅃㅛㄴ ㄲㅛㅅ ㅉㅜㅇ ㅉㅔㅇㅣㄹ ㄱㅣㅃㅠㄴ ㄴㅑㅃㅑㅆㄸㅕㄴ ㄲㅛㅅ.
자모: ㅃㅕㄹ ㅎㅑㄴ ㅋㅐㅋㄸㅛㄶ ㅇㅑㄲㅏㅂㄸㅑㄵ. ㅇㅙ ㅆㅏㅎㄹㅏㅁㄸㅡㅄㄹㅓ ㅃㅕㄹ 1ㄲㅐㄹㅡㄹ ㅉㅠㅆㄴㅡㄴㅉㅣ ㄲㅕㄲㅕㅃㅛㄴ ㅆㅑㄹㅏㅁㅡㄹㅛㅆㅅㅕㄳ ㅁㅑㄹㄹㅗ ㅆㅕㄹㅁㅕㅇㅎㅑㅉㅑㄴㅓ ㄸㅐㅅㄱㅡㄹㄹㅗㄴㅡㄴ ㄴㅓㅁㅠ ㄲㅓㄹㄲㅛㄵ... ㅇㅏㅁㅠㄷㅡㄴ ㄸㅜ ㅃㅕㄴ ㄸㅏㅆㅏㅆ ㄲㅑㄲㅓㄹ ㅆㅓㅀㅇㅡㄴ ㄲㅛㅅ. ㅋㅐㅁㅂㅣㅇㅡㄹ 20ㅇㅕ ㄴㅕㄴ ㄸㅏㄴㅕㅃㅛㄴ ㄲㅛㅅ ㅉㅜㅇ ㅉㅔㅇㅣㄹ ㄱㅣㅃㅠㄴ ㄴㅑㅃㅑㅆㄸㅕㄴ ㄲㅛㅅ.
정답: 별 한 개도 아깝다. 왜 사람들이 별 1개를 주는지 겪어본 사람으로서 말로 설명하자니 댓글로는 너무 길고... 아무튼 두 번 다시 가길 싫은 곳. 캠핑을 20여 년 다녀본 곳 중 제일 기분 나빴던 곳.


In [11]:
# ---------------------------------------------------------
# Train(90%) / Test(10%) 데이터 분리
# ---------------------------------------------------------
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# 인덱스 재정렬
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

print(f"✅ 전체 데이터 개수: {len(train_df) + len(test_df)}개")
print(f"   - 학습용 데이터(Train): {len(train_df)}개")
print(f"   - 테스트용 데이터(Test): {len(test_df)}개")

✅ 전체 데이터 개수: 16169개
   - 학습용 데이터(Train): 12935개
   - 테스트용 데이터(Test): 3234개


In [12]:
# ==========================================
# 2. 데이터셋 및 모델 설정
# ==========================================

KOR_PROMPT = (
    "아래 입력은 난독화된 한국어 문장을 자모(초성/중성/종성)로 분해한 문자열이야.\n"
    "이 자모들을 한국어 문장 그대로 복원해.\n"
)

class KoBARTDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=512):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        raw_input_no_prompt = str(row["input"])   # ✅ 프롬프트 없는 원본
        target_text = str(row['output']) + self.tokenizer.eos_token

        input_text = KOR_PROMPT + raw_input_no_prompt + self.tokenizer.eos_token
        inputs = self.tokenizer(input_text, max_length=self.max_len, padding='max_length', truncation=True, return_tensors='pt')
        targets = self.tokenizer(target_text, max_length=self.max_len, padding='max_length', truncation=True, return_tensors='pt')

        labels = targets["input_ids"].squeeze()
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': labels,
            "raw_input": input_text,                  # 프롬프트 포함
            "raw_input_no_prompt": raw_input_no_prompt, # ✅ 프롬프트 제외
            'raw_output': target_text
        }

print("🔄 토크나이저 및 모델 로딩 중...")
tokenizer = get_kobart_tokenizer()
model = BartForConditionalGeneration.from_pretrained(get_pytorch_kobart_model())
model.to(device)

# 학습용 데이터셋 생성
train_dataset = KoBARTDataset(train_df, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# 테스트용 데이터셋 생성
test_dataset = KoBARTDataset(test_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

🔄 토크나이저 및 모델 로딩 중...
downloading model from https://huggingface.co/skt/kobart-base-v1/resolve/main/legacy/kobart_base_tokenizer_cased_cf74400bce.zip...
downloading model from https://huggingface.co/skt/kobart-base-v1/resolve/main/legacy/kobart_base_cased_ff4bda5738.zip...


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [13]:
# ==========================================
# 3. 학습 (Train Loop)
# ==========================================
optimizer = AdamW(model.parameters(), lr=3e-5)
epochs = 10

print(f"🚀 학습 시작 (총 {epochs} Epochs)...")
model.train()
for epoch in range(epochs):
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
    total_loss = 0
    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        raw_texts = list(batch["raw_input"]) # 배치의 원문 리스트
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    print(f"Epoch {epoch+1} 평균 Loss: {total_loss / len(train_loader):.4f}")

🚀 학습 시작 (총 10 Epochs)...


Epoch 1/10: 100%|██████████| 809/809 [06:12<00:00,  2.17it/s, loss=1.29]


Epoch 1 평균 Loss: 2.5375


Epoch 2/10: 100%|██████████| 809/809 [06:11<00:00,  2.18it/s, loss=0.629]


Epoch 2 평균 Loss: 0.8734


...

Epoch 8 평균 Loss: 0.1644


Epoch 9/10: 100%|██████████| 809/809 [06:11<00:00,  2.18it/s, loss=0.151]


Epoch 9 평균 Loss: 0.1353


Epoch 10/10: 100%|██████████| 809/809 [06:11<00:00,  2.17it/s, loss=0.0389]

Epoch 10 평균 Loss: 0.1121


In [15]:
# ==========================================
# 4. 모델 저장 및 테스트 결과 확인
# ==========================================
# 1) 학습된 모델 저장
model_save_path = '/content/drive/MyDrive/FinalModel1_3/'
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)
print(f"💾 모델 저장 완료: {model_save_path}")

# 2) 테스트 데이터(Test Set)로 결과 생성
print(f"📝 테스트 데이터({len(test_df)}개)에 대한 결과 생성 중...")
model.eval()

result_data = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Inference"):
        input_ids = batch['input_ids'].to(device)

        #✅ 1. 프롬프트 제외 input 길이 계산
        no_prompt_ids = tokenizer(
            list(batch["raw_input_no_prompt"]),
             padding=True,
            truncation=True,
             return_tensors="pt"
         )["input_ids"]

        no_prompt_lens = (no_prompt_ids != tokenizer.pad_token_id).sum(dim=1)

        # 배치 기준 cap (가장 짧은 샘플 기준)
        cap = int(no_prompt_lens.max().item())

        # 안전장치
        max_new_tokens = max(1, cap)

        # 모델 생성 (Beam Search)
        generated_ids = model.generate(
            input_ids=input_ids,
            attention_mask=batch["attention_mask"].to(device),
            num_beams=5,
            eos_token_id=tokenizer.eos_token_id,
            max_new_tokens = max(1, cap),   # 생성 토큰 상한
            early_stopping=False,
            length_penalty=1.2
        )

        generated_texts = tokenizer.batch_decode(
            generated_ids, skip_special_tokens=True
        )

        for i in range(len(generated_texts)):
            result_data.append({
                "input": batch["raw_input"][i],
                "ground_truth": batch["raw_output"][i],
                "prediction": generated_texts[i]
            })

# 결과 저장
df_result = pd.DataFrame(result_data)
output_path = "/content/drive/MyDrive/FinalModel1_3/output_test_results.csv"
df_result.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"✅ 모든 과정 완료! 테스트 결과 파일: {output_path}")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}


💾 모델 저장 완료: /content/drive/MyDrive/FinalModel1_3/
📝 테스트 데이터(3234개)에 대한 결과 생성 중...


Inference: 100%|██████████| 102/102 [10:18<00:00,  6.06s/it]

✅ 모든 과정 완료! 테스트 결과 파일: /content/drive/MyDrive/FinalModel1_3/output_test_results.csv


In [16]:
import pandas as pd
from jiwer import cer


file_path = '/content/drive/MyDrive/FinalModel1_3/output_test_results.csv'
df = pd.read_csv(file_path) # CSV라면 pd.read_csv(file_path)

def cut_to_same_length(pred, ref):

    L = min(len(pred), len(ref))
    return pred[:L], ref[:L]

preds_cut = []
refs_cut = []

for p, r in zip(df["prediction"], df["ground_truth"]):
    pc, rc = cut_to_same_length(p, r)
    preds_cut.append(pc)
    refs_cut.append(rc)

cer_score = cer(refs_cut, preds_cut)

print(f"Truncated CER Score: {cer_score:.4f}")

Truncated CER Score: 0.0579


In [17]:
from bert_score import score

refs_original = df['ground_truth'].tolist()
preds_original = df['prediction'].tolist()

P, R, F1 = score(preds_original, refs_original, lang='ko', verbose=True)

print(f"BERTScore F1 (Average): {F1.mean().item():.4f}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/101 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/51 [00:00<?, ?it/s]

done in 14.68 seconds, 220.36 sentences/sec
BERTScore F1 (Average): 0.9401
